In [1]:
import tensorflow as tf
import time
import numpy as np
from qopt import *
from qopt.solver_algorithms import Solver

C:\Users\julia\Documents\qopt\qopt\matrix.py:80: UserWarning: Qutip not installed. plot_bloch_vector_evolution() is not available
  warn('Qutip not installed. plot_bloch_vector_evolution() is not available')
C:\Users\julia\.conda\envs\qopt_tensorflow\lib\site-packages\filter_functions\plotting.py:68: UserWarning: Qutip not installed. plot_bloch_vector_evolution() is not available
  warn('Qutip not installed. plot_bloch_vector_evolution() is not available')
C:\Users\julia\Documents\qopt\qopt\optimize.py:88: UserWarning: simanneal not installed. SimulatedAnnealing is not available
  warn('simanneal not installed. '
C:\Users\julia\Documents\qopt\qopt\plotting.py:55: UserWarning: Qutip not installed. plot_bloch_vector_evolution() is not available
  warn('Qutip not installed. plot_bloch_vector_evolution() is not available')


In [2]:
class TensorflowSS(Solver):
    def __init__(self):
        pass

In [3]:
n_time_steps = int(1e4)
delta_t = np.pi / n_time_steps

solver = SchroedingerSolver(
    h_ctrl=[.5 * DenseOperator.pauli_x()],
    h_drift=[0 * DenseOperator.pauli_x()],
    tau=delta_t * np.ones(1),
    transfer_function=OversamplingTF(oversampling=n_time_steps, num_ctrls=1)
)

In [4]:
solver.set_optimization_parameters(np.ones((1, 1)))
solver._jit_compute_propagation()

start = time.time()
solver._jit_compute_propagation()
end = time.time()
print(end- start)

C:\Users\julia\Documents\qopt\qopt\solver_algorithms.py:977: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "_jit_compute_propagation" failed type inference due to: Untyped global name 'super': Cannot determine Numba type of <class 'type'>

File "solver_algorithms.py", line 982:
    def _jit_compute_propagation(
        <source elided>
        """See base class. """
        super()._compute_propagation()
        ^

  @numba.jit(nopython=False)


ValueError: Failed in object mode pipeline (step: object mode frontend)
too many values to unpack (expected 1)

In [ ]:


start = time.time()
solver._compute_propagation()
end = time.time()
print(end- start)